# Install meridian: from PyPI @ latest release
!pip install --upgrade google-meridian[colab,and-cuda]

# Install meridian: from PyPI @ specific version
# !pip install google-meridian[colab,and-cuda]==1.0.3

# Install meridian: from GitHub @HEAD
# !pip install --upgrade "google-meridian[colab,and-cuda] @ git+https://github.com/google/meridian.git"

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_probability as tfp
import arviz as az

import IPython

from meridian import constants
from meridian.data import load
from meridian.data import test_utils
from meridian.model import model
from meridian.model import spec
from meridian.model import prior_distribution
from meridian.analysis import optimizer#
from meridian.analysis import analyzer
from meridian.analysis import visualizer
from meridian.analysis import summarizer
from meridian.analysis import formatter

# check if GPU is available
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print("Num CPUs Available: ", len(tf.config.experimental.list_physical_devices('CPU')))




Your runtime has 16.6 gigabytes of available RAM

Num GPUs Available:  0
Num CPUs Available:  1


In [ ]:

import io
import requests


coord_to_columns = load.CoordToColumns(
    time='Date',
    #geo='geo',
    controls=['seasonality_sin' ,'seasonality_cos','is_holiday_week','Organic_impressions','Organic_impressions_lag1','avg_temp','rain_mm'],
    #population='population',
    kpi='total_revenue',
    #revenue_per_kpi='revenue_per_conversion',
    media=[
        'facebook_impressions',
        'bing_impressions',
        'google_impressions',
        'tiktok_impressions',
        'pinterest_impressions',
    ],
    media_spend=[
       'bing_spend',
      'pinterest_spend',
      'facebook_spend',
      'google_spend',
      'tiktok_spend',
    ],
   # organic_media=['Organic_impressions'],
    #non_media_treatments=['Promo'],
)

correct_media_to_channel = {
    'facebook_impressions': 'Facebook',
    'bing_impressions': 'Bing',
    'google_impressions': 'Google',
    'tiktok_impressions': 'Tiktok',
    'pinterest_impressions': 'Pinterest',
}
correct_media_spend_to_channel = {
    'bing_spend': 'Bing',
    'pinterest_spend': 'Pinterest',
    'facebook_spend': 'Facebook',
    'google_spend': 'Google',
    'tiktok_spend': 'Tiktok'
}
# Download the CSV data
url = 'https://raw.githubusercontent.com/Connective3Innovation/impact_V2/refs/heads/main/Data/charles_ivy_with_weather_added_v3.csv' # Updated to raw content URL
response = requests.get(url)
response.raise_for_status() # Raise HTTPError for bad responses (4xx or 5xx)
csv_data = response.content.decode('utf-8') # Assuming UTF-8 encoding

# Load into pandas DataFrame
df = pd.read_csv(io.StringIO(csv_data), sep=',') # Explicitly setting separator and skipping bad lines

loader = load.DataFrameDataLoader(
    #csv_path='https://github.com/Connective3Innovation/impact_V2/blob/main/Data/charles%26ivy_apr2023_apr_2025_cleaned.csv',
    df=df, # Pass the DataFrame directly
    kpi_type='revenue',
    coord_to_columns=coord_to_columns,
    media_to_channel=correct_media_to_channel,
    media_spend_to_channel=correct_media_spend_to_channel,
)
data = loader.load()

c:\Users\GagandeepBajwa\Documents\impact_V2\venv\Lib\site-packages\meridian\data\load.py:1023: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.df[geo_column_name] = self.df[geo_column_name].replace(
c:\Users\GagandeepBajwa\Documents\impact_V2\venv\Lib\site-packages\meridian\data\load.py:201: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  if (constants.GEO) not in self.dataset.dims.keys():
c:\Users\GagandeepBajwa\Documents\impact_V2\venv\Lib\site-packages\meridian\data\load.py:206: UserWarning: The `population` argument is ignored in a nationally aggregated m

In [3]:
roi_mu = 0.2     # Mu for ROI prior for each media channel.
roi_sigma = 0.9  # Sigma for ROI prior for each media channel.
prior = prior_distribution.PriorDistribution(
    roi_m=tfp.distributions.LogNormal(roi_mu, roi_sigma, name=constants.ROI_M)
)
model_spec = spec.ModelSpec(prior=prior)

mmm = model.Meridian(input_data=data, model_spec=model_spec)

c:\Users\GagandeepBajwa\Documents\impact_V2\venv\Lib\site-packages\meridian\model\model.py:64: UserWarning: In a nationally aggregated model, the `media_effects_dist` will be reset to `normal`.
  warnings.warn(


In [4]:
df.loc[df['total_revenue'].idxmax()]

Date                        2024-11-25
facebook_impressions           1820099
bing_impressions                 91637
google_impressions             3268585
tiktok_impressions             95397.0
pinterest_impressions           119031
bing_spend                 1913.109997
pinterest_spend            1057.043594
facebook_spend            23655.069995
google_spend              41471.219546
tiktok_spend                570.849999
total_revenue              444492.5741
total_conversions              8303.12
facebook_revenue         147751.099998
tiktok_revenue                  6703.0
pinterest_revenue             17649.35
Organic_impressions           148874.0
bing_revenue                  43815.59
google_revenue           228573.534075
week_of_year                        48
seasonality_sin              -0.464723
seasonality_cos               0.885456
is_holiday_week                      0
geo                       national_geo
population                         1.0
Name: 87, dtype: object

In [4]:
%%time
mmm.sample_prior(500)
mmm.sample_posterior(n_chains=10, n_adapt=2000, n_burnin=500, n_keep=1000, seed=1)

c:\Users\GagandeepBajwa\Documents\impact_V2\venv\Lib\site-packages\meridian\model\prior_distribution.py:852: UserWarning: Hierarchical distribution parameters must be deterministically zero for national models. tau_g_excl_baseline has been automatically set to Deterministic(0).
  warnings.warn(
c:\Users\GagandeepBajwa\Documents\impact_V2\venv\Lib\site-packages\meridian\model\prior_distribution.py:852: UserWarning: Hierarchical distribution parameters must be deterministically zero for national models. eta_m has been automatically set to Deterministic(0).
  warnings.warn(
c:\Users\GagandeepBajwa\Documents\impact_V2\venv\Lib\site-packages\meridian\model\prior_distribution.py:852: UserWarning: Hierarchical distribution parameters must be deterministically zero for national models. eta_rf has been automatically set to Deterministic(0).
  warnings.warn(
c:\Users\GagandeepBajwa\Documents\impact_V2\venv\Lib\site-packages\meridian\model\prior_distribution.py:852: UserWarning: Hierarchical dist

CPU times: total: 4min 10s
Wall time: 4min 22s


In [5]:
model_diagnostics = visualizer.ModelDiagnostics(mmm)
model_diagnostics.plot_rhat_boxplot()

alt.LayerChart(...)

In [6]:
model_fit = visualizer.ModelFit(mmm)
model_fit.plot_model_fit()

c:\Users\GagandeepBajwa\Documents\impact_V2\venv\Lib\site-packages\meridian\analysis\analyzer.py:540: UserWarning: The `aggregate_geos` argument is ignored in the national model. It will be reset to `True`.
  warnings.warn(


alt.LayerChart(...)

In [7]:
mmm_summarizer = summarizer.Summarizer(mmm)

In [8]:
import os
filepath = os.path.join(os.getcwd())
start_date = '2023-03-27'
end_date = '2025-04-28' # Changed end_date to a valid date
mmm_summarizer.output_model_results_summary('summary_output_charlesivy_with_organic_lag_v4.html', filepath, start_date, end_date)

c:\Users\GagandeepBajwa\Documents\impact_V2\venv\Lib\site-packages\meridian\analysis\analyzer.py:540: UserWarning: The `aggregate_geos` argument is ignored in the national model. It will be reset to `True`.
  warnings.warn(
c:\Users\GagandeepBajwa\Documents\impact_V2\venv\Lib\site-packages\meridian\analysis\analyzer.py:540: UserWarning: The `aggregate_geos` argument is ignored in the national model. It will be reset to `True`.
  warnings.warn(
c:\Users\GagandeepBajwa\Documents\impact_V2\venv\Lib\site-packages\meridian\analysis\analyzer.py:3063: UserWarning: Effectiveness is not reported because it does not have a clear interpretation by time period.
  warnings.warn(
c:\Users\GagandeepBajwa\Documents\impact_V2\venv\Lib\site-packages\meridian\analysis\analyzer.py:925: UserWarning: Setting `use_kpi=True` has no effect when `kpi_type=REVENUE` since in this case, KPI is equal to revenue.
  warnings.warn(
c:\Users\GagandeepBajwa\Documents\impact_V2\venv\Lib\site-packages\tensorflow\python\au

In [9]:
with open('summary_output_charlesivy_with_organic_lag_v4.html', 'r', encoding='utf-8') as file:
    report_content = file.read()

report_content = report_content.replace('$', '£')

with open('summary_output_charlesivy_with_organic_lag_v4.html', 'w', encoding='utf-8') as file:
    file.write(report_content)

In [10]:
%%time
budget_optimizer = optimizer.BudgetOptimizer(mmm)
optimization_results = budget_optimizer.optimize()

CPU times: total: 5min 5s
Wall time: 3min 37s


In [11]:
optimization_results.output_optimization_summary('optimization_output_charlesivy_v4.html', filepath)
with open('optimization_output_charlesivy_v4.html', 'r', encoding='utf-8') as file:
    report_content = file.read()

report_content = report_content.replace('$', '£')

with open('optimization_output_charlesivy_v4.html', 'w', encoding='utf-8') as file:
    file.write(report_content)

In [12]:
save_path=os.path.join(os.getcwd(),'saved_mmm_charlesivy_v4.pkl')
model.save_mmm(mmm, save_path)

In [14]:
model.prior_distribution

<module 'meridian.model.prior_distribution' from 'c:\\Users\\GagandeepBajwa\\Documents\\impact_V2\\venv\\Lib\\site-packages\\meridian\\model\\prior_distribution.py'>

In [16]:
import requests
import pandas as pd

# Set your API key here
API_KEY = '9HLPFRULYNU5ZDAHYGRAMQ9QE'
LOCATION = 'United Kingdom'
START_DATE = '2023-03-27'
END_DATE = '2025-05-05'

# Build the API URL
url = f'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{LOCATION}/{START_DATE}/{END_DATE}?unitGroup=metric&key={API_KEY}&include=days'

# Make the request
response = requests.get(url)
print(response.status_code)
print(response.text)

data = response.json()

# Extract daily data
daily_data = pd.DataFrame(data['days'])
daily_data['datetime'] = pd.to_datetime(daily_data['datetime'])

# Add a "week start" column
daily_data['week_start'] = daily_data['datetime'] - pd.to_timedelta(daily_data['datetime'].dt.dayofweek, unit='d')

# Aggregate by week
weekly_weather = daily_data.groupby('week_start').agg({
    'temp': 'mean',
    'precip': 'sum'
}).reset_index()

# Rename columns
weekly_weather.columns = ['Date', 'avg_temp', 'rain_mm']

# Display sample
print(weekly_weather.head())


429
You have exceeded the maximum number of daily result records for your account. 


JSONDecodeError: Expecting value: line 1 column 1 (char 0)